<a href="https://colab.research.google.com/github/ykitaguchi77/GravCont_classification_colab/blob/master/%E3%83%87%E3%83%BC%E3%82%BF%E3%82%BB%E3%83%83%E3%83%88%E3%82%92%E3%83%A9%E3%83%B3%E3%83%80%E3%83%A0%E3%81%AB%E6%8C%AF%E3%82%8A%E5%88%86%E3%81%91.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**GravCont_Resnet_random**

In [1]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.utils.data as data
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
import math
import shutil

#Advanced Pytorchから
import glob
import os.path as osp
import random
import json
from PIL import Image
from tqdm import tqdm
%matplotlib inline

#サポートパッチのインポート
from google.colab.patches import cv2_imshow
import cv2

plt.ion()   # interactive mode
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Set random seem for reproducibility
manualSeed = 20200815
#manualSeed = random.randint(1, 10000) # use if you want new results
print("Random Seed: ", manualSeed)
random.seed(manualSeed)
torch.manual_seed(manualSeed)
torch.cuda.manual_seed(manualSeed)

torch.torch.backends.cudnn.benchmark = True
torch.torch.backends.cudnn.enabled = True


'''
grav: 甲状腺眼症
cont: コントロール
黒の空白を挿入することにより225px*225pxの画像を生成、EfficientNetを用いて転移学習
－－－－－－－－－－－－－－
データの構造
gravcont.zip ------grav
               |---cont
'''                                     

#google driveをcolabolatoryにマウント
from google.colab import drive
drive.mount('/content/drive')

Random Seed:  20200815
Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
4/3wGH7XearU95TA03Mw3hHXqgskoIcdt4BGmhw6eN4tVJIBAkA4knHOo
Mounted at /content/drive


#**データセットをランダムに振り分け**

In [3]:
#処理時間の計測
start = time.time()

path = '/content/drive/My Drive/Grav_bootcamp/PrePlusTrain_2/'
seed_path = '/content/drive/My Drive/Grav_bootcamp/PrePlusTrain_2_seed_'+str(manualSeed)

if os.path.exists(seed_path):
    shutil.rmtree(seed_path)
os.makedirs(seed_path, exist_ok=True)

classes = os.listdir(path)
print(classes)


list = [0] * len(classes)
train = [0] * len(classes)
val = [0] * len(classes)
k=0
for i in classes:
    list[k] = os.listdir(path + i)
    train[k] = random.sample(list[k], int(len(list[k])*0.8))
    val[k] = [x for x in list[k] if x not in train[k]]
    os.makedirs(seed_path +'/train/'+i, exist_ok=True) #trainのフォルダを作成
    os.makedirs(seed_path +'/val/'+i, exist_ok=True) #valのフォルダを作成
    for j in train[k]:
          shutil.copy(path+i+'/'+j, seed_path +'/train/'+i)
    for j in val[k]:
          shutil.copy(path+i+'/'+j, seed_path +'/val/'+i)

print('Process done!!')
elapsed_time = time.time() - start
print ("elapsed_time:{0}".format(elapsed_time) + "[sec]")

['grav', 'cont']
Process done!!
elapsed_time:265.53483963012695[sec]


#**フォルダ内容の確認**

In [4]:
print('train:')
print('  grav: '+str(len(glob.glob(seed_path+'/train/grav/*'))))
print('  cont: '+str(len(glob.glob(seed_path+'/train/cont/*'))))
print('val:')
print('  grav: '+str(len(glob.glob(seed_path+'/val/grav/*'))))
print('  cont: '+str(len(glob.glob(seed_path+'/val/cont/*'))))

train:
  grav: 223
  cont: 223
val:
  grav: 56
  cont: 56


#フォルダとその中のファイルを削除するスクリプト

In [ ]:
#フォルダを削除するスクリプト
directory = '/content/drive/My Drive/Deep_learning/gravcont_seed_1234'
files = os.listdir(directory)
print(directory)
print(files)

for i in files:
    try:
        os.remove(directory+'/'+i)
    except IsADirectoryError:
        pass

shutil.rmtree(directory)